# Partie sur la prediction des données

## Prédiction des scores des elections du premier tour 2022

### Bibliotheque

In [1]:
import pandas as pd
import numpy as np
import random
import os
import statsmodels.api as sm
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score
from scipy.stats import pearsonr
from sklearn.metrics import r2_score

### Préparation des données

In [2]:
# Sélection des features

df_concat = pd.read_excel('Commune.xlsx')
df_concat = df_concat.sample(frac=1, random_state=42)
    
# Sélection des features
features = df_concat.drop(['COM','Macron', 'Lepen', 'Abstentions','ABSTENTIONS', 'ARTHAUD', 'ROUSSEL', 'MACRON', 'LASSALLE', 'LEPEN', 'ZEMMOUR', 'MÉLENCHON', 'HIDALGO', 'JADOT', 'PÉCRESSE', 'POUTOU', 'DUPONT-AIGNAN'], axis=1)

Classes = ['ARTHAUD', 'ROUSSEL', 'MACRON', 'LASSALLE', 'LEPEN', 'ZEMMOUR', 'MÉLENCHON', 'HIDALGO', 'JADOT', 'PÉCRESSE', 'POUTOU', 'DUPONT-AIGNAN']
y = df_concat[Classes]

class_mapping = {
    0: 'ARTHAUD',
    1: 'ROUSSEL',
    2: 'MACRON',
    3: 'LASSALLE',
    4: 'LEPEN',
    5: 'ZEMMOUR',
    6: 'MÉLENCHON',
    7: 'HIDALGO',
    8: 'JADOT',
    9: 'PÉCRESSE',
    10: 'POUTOU',
    11: 'DUPONT-AIGNAN'
}



# Normalisation des données
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(features)

scalery = MinMaxScaler()
y_scaled = scalery.fit_transform(y)


###  Selection des Colonnes intéressante

In [3]:
def Selection_Colonne_Pvalue(label):
    # Création de l'analyse pour la p_value
    ListeSignificant_columns=[]
    for y in label.T:
        p_values = {}
        for i, column in enumerate(features.columns):
            p_value = pearsonr(X_scaled[:, i], y.flatten())[1]
            p_values[i] = p_value

        # Sélectionnez les colonnes avec des p-values inférieures au seuil de significativité
        significant_indices = [column for column, p_value in p_values.items() if p_value < 0.05]
        print("Affichage des colonnes selectionné : ",len(features.iloc[:, significant_indices].columns))
        ListeSignificant_columns.append(X_scaled[:, significant_indices])
    return ListeSignificant_columns

X_scaled_Significant = Selection_Colonne_Pvalue(y_scaled)

#division des données
split_index1 = int(df_concat.shape[0] * 0.8)
split_index2 = int(df_concat.shape[0] * 0.9)

def creationSLIP(X_scaled_Significant):
    X_train_classe=[]
    X_val_classe=[]
    X_test_classe=[]

    for X in X_scaled_Significant:
        X_train, X_val, X_test = X[:split_index1, :], X[split_index1:split_index2, :], X[split_index2:, :]

        X_train_classe.append(X_train)
        X_val_classe.append(X_val)
        X_test_classe.append(X_test)
    return X_train_classe, X_val_classe, X_test_classe

X_train_classe, X_val_classe, X_test_classe = creationSLIP(X_scaled_Significant)
y_train, y_val, y_test = y_scaled[:split_index1], y_scaled[split_index1:split_index2], y_scaled[split_index2:]

print(len(X_train_classe))
print(y_train.shape)

Affichage des colonnes selectionné :  25
Affichage des colonnes selectionné :  27
Affichage des colonnes selectionné :  29
Affichage des colonnes selectionné :  33
Affichage des colonnes selectionné :  33
Affichage des colonnes selectionné :  30
Affichage des colonnes selectionné :  30
Affichage des colonnes selectionné :  32
Affichage des colonnes selectionné :  33
Affichage des colonnes selectionné :  29
Affichage des colonnes selectionné :  25
Affichage des colonnes selectionné :  31
12
(25085, 12)


### Application de Regression

Macron

In [4]:
from sklearn.linear_model import LinearRegression

def LinearRegression_classe(X_train_classe, X_val_classe, X_test_classe,y_train, y_val, y_test):
    
    for i in range (len(X_train_classe)):
        print("Prediction pour ", class_mapping[i])
        linear_reg = LinearRegression()
        linear_reg.fit(X_train_classe[i], y_train[:, i])

        predictions_logistic = linear_reg.predict(X_test_classe[i])
        correlation, p_value = pearsonr(y_test[:, i].ravel(), predictions_logistic)

        print("Corrélation entre les valeurs réelles et prédites :", correlation)
        print("P-value associée :", p_value)
        print("R-value :", r2_score(y_test[:, i].ravel(), predictions_logistic))
        print()
        
LinearRegression_classe(X_train_classe, X_val_classe, X_test_classe,y_train, y_val, y_test)

Prediction pour  ARTHAUD
Corrélation entre les valeurs réelles et prédites : 0.17655072317398504
P-value associée : 2.252309587562438e-23
R-value : 0.0305560152063048

Prediction pour  ROUSSEL
Corrélation entre les valeurs réelles et prédites : 0.2590680887695219
P-value associée : 2.8724770565273994e-49
R-value : 0.06684334314270113

Prediction pour  MACRON
Corrélation entre les valeurs réelles et prédites : 0.6517261495940352
P-value associée : 0.0
R-value : 0.4246464431906751

Prediction pour  LASSALLE
Corrélation entre les valeurs réelles et prédites : 0.5209327625240026
P-value associée : 9.64644864803579e-218
R-value : 0.2711353507616244

Prediction pour  LEPEN
Corrélation entre les valeurs réelles et prédites : 0.8871495957791171
P-value associée : 0.0
R-value : 0.7868635828604205

Prediction pour  ZEMMOUR
Corrélation entre les valeurs réelles et prédites : 0.4713418889235785
P-value associée : 3.1791777404847035e-173
R-value : 0.22173311223269376

Prediction pour  MÉLENCHON
Cor

In [5]:
from sklearn.linear_model import SGDRegressor

def SGDRegressor_classe(X_train_classe, X_val_classe, X_test_classe,y_train, y_val, y_test):
    
    for i in range (len(X_train_classe)):
        print("Prediction pour ", class_mapping[i])
        
        linear_reg = SGDRegressor()
        linear_reg.fit(X_train_classe[i], y_train[:, i])

        predictions_logistic = linear_reg.predict(X_test_classe[i])
        correlation, p_value = pearsonr(y_test[:, i].ravel(), predictions_logistic)

        print("Corrélation entre les valeurs réelles et prédites :", correlation)
        print("P-value associée :", p_value)
        print("R-value :", r2_score(y_test[:, i].ravel(), predictions_logistic))
        print()
        
SGDRegressor_classe(X_train_classe, X_val_classe, X_test_classe,y_train, y_val, y_test)

Prediction pour  ARTHAUD
Corrélation entre les valeurs réelles et prédites : 0.17932403162094626
P-value associée : 4.491256789602388e-24
R-value : 0.030547695565887545

Prediction pour  ROUSSEL
Corrélation entre les valeurs réelles et prédites : 0.2583322371354862
P-value associée : 5.45967091092327e-49
R-value : 0.06291532992504023

Prediction pour  MACRON
Corrélation entre les valeurs réelles et prédites : 0.599039060621323
P-value associée : 7.660536140600125e-305
R-value : 0.35153777730617075

Prediction pour  LASSALLE
Corrélation entre les valeurs réelles et prédites : 0.4814807452222653
P-value associée : 9.728566675351884e-182
R-value : 0.22292708427237096

Prediction pour  LEPEN
Corrélation entre les valeurs réelles et prédites : 0.8855565521801683
P-value associée : 0.0
R-value : 0.7823815628214217

Prediction pour  ZEMMOUR
Corrélation entre les valeurs réelles et prédites : 0.4632876891816323
P-value associée : 1.1592821858474586e-166
R-value : 0.197879913269298

Prediction 

In [6]:
from sklearn.linear_model import Ridge

def Ridge_classe(X_train_classe, X_val_classe, X_test_classe,y_train, y_val, y_test):
    
    for i in range (len(X_train_classe)):
        print("Prediction pour ", class_mapping[i])
        
        linear_reg = Ridge()
        linear_reg.fit(X_train_classe[i], y_train[:, i])

        predictions_logistic = linear_reg.predict(X_test_classe[i])
        correlation, p_value = pearsonr(y_test[:, i].ravel(), predictions_logistic)

        print("Corrélation entre les valeurs réelles et prédites :", correlation)
        print("P-value associée :", p_value)
        print("R-value :", r2_score(y_test[:, i].ravel(), predictions_logistic))
        print()
        
Ridge_classe(X_train_classe, X_val_classe, X_test_classe,y_train, y_val, y_test)

Prediction pour  ARTHAUD
Corrélation entre les valeurs réelles et prédites : 0.17711535448022708
P-value associée : 1.6254892481256135e-23
R-value : 0.030792458758243324

Prediction pour  ROUSSEL
Corrélation entre les valeurs réelles et prédites : 0.25922636011260236
P-value associée : 2.50120543210428e-49
R-value : 0.06689963526243015

Prediction pour  MACRON
Corrélation entre les valeurs réelles et prédites : 0.6515779205433494
P-value associée : 0.0
R-value : 0.42445611226607616

Prediction pour  LASSALLE
Corrélation entre les valeurs réelles et prédites : 0.521013704221824
P-value associée : 8.044978931948585e-218
R-value : 0.2710814832704831

Prediction pour  LEPEN
Corrélation entre les valeurs réelles et prédites : 0.8871783907351356
P-value associée : 0.0
R-value : 0.7869149817124494

Prediction pour  ZEMMOUR
Corrélation entre les valeurs réelles et prédites : 0.4716112775261544
P-value associée : 1.9045175334689907e-173
R-value : 0.2220531690963904

Prediction pour  MÉLENCHON
C

### Foret Aleatoire

In [7]:
from sklearn.ensemble import RandomForestRegressor

def RandomForest_classe(X_train_classe, X_val_classe, X_test_classe,y_train, y_val, y_test):
    
    for i in range (len(X_train_classe)):
        print("Prediction pour ", class_mapping[i])
        
        rf_regressor = RandomForestRegressor(n_estimators=100) 
        rf_regressor.fit(X_train_classe[i], y_train[:, i])

        predictions_rf = rf_regressor.predict(X_test_classe[i])
        correlation, p_value = pearsonr(y_test[:, i].ravel(), predictions_rf)

        print("Corrélation entre les valeurs réelles et prédites :", correlation)
        print("P-value associée :", p_value)
        print("R-value :", r2_score(y_test[:, i].ravel(), predictions_rf))
        print()
        
RandomForest_classe(X_train_classe, X_val_classe, X_test_classe,y_train, y_val, y_test)

Prediction pour  ARTHAUD
Corrélation entre les valeurs réelles et prédites : 0.14827688429324004
P-value associée : 7.039305617657216e-17
R-value : 0.005692156837824647

Prediction pour  ROUSSEL
Corrélation entre les valeurs réelles et prédites : 0.2710468953527726
P-value associée : 6.173427754322846e-54
R-value : 0.06809301898975917

Prediction pour  MACRON
Corrélation entre les valeurs réelles et prédites : 0.6952567892064724
P-value associée : 0.0
R-value : 0.4821542655078537

Prediction pour  LASSALLE
Corrélation entre les valeurs réelles et prédites : 0.6162117003156724
P-value associée : 0.0
R-value : 0.3786697979586269

Prediction pour  LEPEN
Corrélation entre les valeurs réelles et prédites : 0.8884010190333967
P-value associée : 0.0
R-value : 0.7891621350873957

Prediction pour  ZEMMOUR
Corrélation entre les valeurs réelles et prédites : 0.48141419753590103
P-value associée : 1.1088542028498191e-181
R-value : 0.22986913519484464

Prediction pour  MÉLENCHON
Corrélation entre l

### SVM

Macron

In [9]:
from sklearn.svm import SVR

def SVR_classe(X_train_classe, X_val_classe, X_test_classe,y_train, y_val, y_test):
    
    for i in range (len(X_train_classe)):
        print("Prediction pour ", class_mapping[i])
        
        rf_regressor = SVR(kernel='linear')
        rf_regressor.fit(X_train_classe[i], y_train[:, i])

        predictions_rf = rf_regressor.predict(X_test_classe[i])
        correlation, p_value = pearsonr(y_test[:, i].ravel(), predictions_rf)

        print("Corrélation entre les valeurs réelles et prédites :", correlation)
        print("P-value associée :", p_value)
        print("R-value :", r2_score(y_test[:, i].ravel(), predictions_rf))
        print()
        
SVR_classe(X_train_classe, X_val_classe, X_test_classe,y_train, y_val, y_test)

Prediction pour  ARTHAUD
Corrélation entre les valeurs réelles et prédites : 0.023241159431154637
P-value associée : 0.19320307925826025
R-value : -0.2148112478355395

Prediction pour  ROUSSEL
Corrélation entre les valeurs réelles et prédites : 0.07974145584807285
P-value associée : 7.790499737072648e-06
R-value : -1.4050671470419918

Prediction pour  MACRON
Corrélation entre les valeurs réelles et prédites : 0.6511241495220759
P-value associée : 0.0
R-value : 0.4234617004222847

Prediction pour  LASSALLE
Corrélation entre les valeurs réelles et prédites : 0.4910712345611994
P-value associée : 4.632054782381413e-190
R-value : -0.1697523152149869

Prediction pour  LEPEN
Corrélation entre les valeurs réelles et prédites : 0.886938421324851
P-value associée : 0.0
R-value : 0.7860181082678688

Prediction pour  ZEMMOUR
Corrélation entre les valeurs réelles et prédites : 0.4670733236189445
P-value associée : 1.0043568396171211e-169
R-value : 0.1933943156382516

Prediction pour  MÉLENCHON
Cor

In [10]:
def SVRPoly_classe(X_train_classe, X_val_classe, X_test_classe,y_train, y_val, y_test):
    
    for i in range (len(X_train_classe)):
        print("Prediction pour ", class_mapping[i])
        
        rf_regressor = SVR(kernel='poly')
        rf_regressor.fit(X_train_classe[i], y_train[:, i])

        predictions_rf = rf_regressor.predict(X_test_classe[i])
        correlation, p_value = pearsonr(y_test[:, i].ravel(), predictions_rf)

        print("Corrélation entre les valeurs réelles et prédites :", correlation)
        print("P-value associée :", p_value)
        print("R-value :", r2_score(y_test[:, i].ravel(), predictions_rf))
        print()
        
SVRPoly_classe(X_train_classe, X_val_classe, X_test_classe,y_train, y_val, y_test)

Prediction pour  ARTHAUD
Corrélation entre les valeurs réelles et prédites : 0.07206446383203288
P-value associée : 5.3621598252575565e-05
R-value : -0.18967383461324183

Prediction pour  ROUSSEL
Corrélation entre les valeurs réelles et prédites : 0.13532265681729952
P-value associée : 2.7393513445009345e-14
R-value : -1.2022484048139694

Prediction pour  MACRON
Corrélation entre les valeurs réelles et prédites : 0.6638699116279521
P-value associée : 0.0
R-value : 0.43817312203786674

Prediction pour  LASSALLE
Corrélation entre les valeurs réelles et prédites : 0.5219229043619348
P-value associée : 1.0433637244449667e-218
R-value : 0.01617752655654303

Prediction pour  LEPEN
Corrélation entre les valeurs réelles et prédites : 0.8844135804009094
P-value associée : 0.0
R-value : 0.7816463226925627

Prediction pour  ZEMMOUR
Corrélation entre les valeurs réelles et prédites : 0.45254421575991477
P-value associée : 3.5263655881641386e-158
R-value : 0.15935044488045813

Prediction pour  MÉLE

### Reseau de neuronne

In [4]:
#conversion pandas -> numpy

def conversionNumpy(X_train_classe, X_val_classe, X_test_classe,y_train, y_val, y_test):
    X_train_classe_new = []
    X_val_classe_new = []
    X_test_classe_new = []
    
    for i in range (len(X_train_classe)):
        X_train_classe_new.append(np.array(X_train_classe[i],dtype=float))
        X_val_classe_new.append(np.array(X_val_classe[i],dtype=float))
        X_test_classe_new.append(np.array(X_test_classe[i],dtype=float))
        
    y_trainNN = np.array(y_train,dtype=float)
    y_valNN = np.array(y_val,dtype=float)
    y_testNN = np.array(y_test,dtype=float)
    return X_train_classe_new, X_val_classe_new, X_test_classe_new, y_trainNN, y_valNN ,y_testNN

X_train_classe_new, X_val_classe_new, X_test_classe_new, y_trainNN, y_valNN ,y_testNN = conversionNumpy(X_train_classe, X_val_classe, X_test_classe,y_train, y_val, y_test)

In [5]:
import tensorflow as tf
from tensorflow.keras import optimizers, losses, callbacks
import numpy as np


CallBackSave = callbacks.ModelCheckpoint("best_model_Valeurs1ertour.keras", 
                                                 monitor='val_loss', 
                                                 save_best_only=True, 
                                                 mode='min')

callbackStop = callbacks.EarlyStopping(monitor='val_loss', 
                                                  patience=20)

def createModel():
    #model = tf.keras.Sequential([tf.keras.layers.Dense(units=1, input_shape=[features.shape[1]])])

    model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(features.shape[1], activation='relu'),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1)
    ])
    model.compile(loss='mean_squared_error',
                  optimizer=optimizers.Adam())
    return model


#### Entrainement pour Macron

In [8]:
from sklearn.svm import SVR

def Train(X_train_classe_new, X_val_classe_new, X_test_classe_new, y_trainNN, y_valNN ,y_testNN):
    
    for i in range (len(X_train_classe)):
        print("Prediction pour ", class_mapping[i])
        model = createModel()
        model.compile(loss='mean_squared_error',
                      optimizer=optimizers.Adam())

        history = model.fit(X_train_classe_new[i], y_trainNN[:,i],
                            epochs=1000,validation_data=(X_val_classe_new[i], y_valNN[:,i]),
                            callbacks=[CallBackSave,callbackStop]
                           )

        y_pred_NN = model.predict(X_test_classe_new[i])

        test_loss = model.evaluate(X_test_classe_new[i], y_testNN[:,i])
        print("Perte de test :", test_loss)
        
        correlation, p_value = pearsonr(np.squeeze(y_testNN[:,i]),  np.squeeze(y_pred_NN))

        # Affichage de la corrélation et de la p-value
        print("Corrélation entre les valeurs réelles et prédites :", correlation)
        print("P-value associée :", p_value)

        print("R-value :", r2_score(np.squeeze(y_testNN[:,i]),  np.squeeze(y_pred_NN)))
        
Train(X_train_classe_new, X_val_classe_new, X_test_classe_new, y_trainNN, y_valNN ,y_testNN)

Prediction pour  ARTHAUD
Epoch 1/1000
784/784 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 0.0040 - val_loss: 0.0037
Epoch 2/1000
784/784 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.0039 - val_loss: 0.0037
Epoch 3/1000
784/784 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.0039 - val_loss: 0.0037
Epoch 4/1000
784/784 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 0.0039 - val_loss: 0.0037
Epoch 5/1000
784/784 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 0.0040 - val_loss: 0.0037
Epoch 6/1000
784/784 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.0038 - val_loss: 0.0037
Epoch 7/1000
784/784 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 0.0039 - val_loss: 0.0038
Epoch 8/1000
784/784 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 0.0040 - val_loss: 0.0037
Epoch 9/1000
784/784 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 0.0037 - val_loss: 0.0037
Epoch 10/1000
784/784 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.0039 - val_loss: 0.0037
Epoch 11/1000
784/784 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.0038 - val_loss: 0.0036
Epoch

Epoch 1/1000
784/784 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.0135 - val_loss: 0.0096
Epoch 2/1000
784/784 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.0078 - val_loss: 0.0075
Epoch 3/1000
784/784 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.0075 - val_loss: 0.0075
Epoch 4/1000
784/784 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.0077 - val_loss: 0.0075
Epoch 5/1000
784/784 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.0074 - val_loss: 0.0072
Epoch 6/1000
784/784 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.0074 - val_loss: 0.0072
Epoch 7/1000
784/784 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.0074 - val_loss: 0.0071
Epoch 8/1000
784/784 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.0072 - val_loss: 0.0071
Epoch 9/1000
784/784 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.0072 - val_loss: 0.0071
Epoch 10/1000
784/784 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.0072 - val_loss: 0.0071
Epoch 11/1000
784/784 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.0071 - val_loss: 0.0072
Epoch 12/1000
784/784 ━━━━━━━━

784/784 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.0020 - val_loss: 0.0021
Epoch 20/1000
784/784 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.0019 - val_loss: 0.0021
98/98 ━━━━━━━━━━━━━━━━━━━━ 0s 696us/step
98/98 ━━━━━━━━━━━━━━━━━━━━ 0s 681us/step - loss: 0.0024
Perte de test : 0.0024039912968873978
Corrélation entre les valeurs réelles et prédites : 0.38992284530181553
P-value associée : 2.1205440529171658e-114
R-value : 0.14680873163926356
Prediction pour  JADOT
Epoch 1/1000
784/784 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.0104 - val_loss: 0.0093
Epoch 2/1000
784/784 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.0090 - val_loss: 0.0095
Epoch 3/1000
784/784 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.0088 - val_loss: 0.0092
Epoch 4/1000
784/784 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.0087 - val_loss: 0.0093
Epoch 5/1000
784/784 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.0086 - val_loss: 0.0094
Epoch 6/1000
784/784 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.0086 - val_loss: 0.0092
Epoch

784/784 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.0045 - val_loss: 0.0050
Epoch 18/1000
784/784 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.0046 - val_loss: 0.0051
Epoch 19/1000
784/784 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.0045 - val_loss: 0.0051
Epoch 20/1000
784/784 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.0046 - val_loss: 0.0050
98/98 ━━━━━━━━━━━━━━━━━━━━ 0s 706us/step
98/98 ━━━━━━━━━━━━━━━━━━━━ 0s 667us/step - loss: 0.0049
Perte de test : 0.0047301617451012135
Corrélation entre les valeurs réelles et prédites : 0.21667484400660364
P-value associée : 1.2330944827981207e-34
R-value : 0.038886014072657016
Prediction pour  DUPONT-AIGNAN
Epoch 1/1000
784/784 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.0053 - val_loss: 0.0053
Epoch 2/1000
784/784 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.0049 - val_loss: 0.0051
Epoch 3/1000
784/784 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.0048 - val_loss: 0.0051
Epoch 4/1000
784/784 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.0047 - val_loss: 0.